In [8]:
%load_ext autoreload
%autoreload 2

import re
import os
import random
import pprint

import sys
sys.path.append('/home/users/pjh/scripts/python_genome_package_dev')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [1]:
import urllib.request
import io
import gzip
import os

In [2]:
import Bio.Seq
import Bio.SeqUtils
import Bio.SeqRecord
import Bio.SeqIO
import Bio.SeqFeature
import Bio.AlignIO
import Bio.Align.Applications

In [3]:
FASTA_PATH_HG19 = '/home/users/data/01_reference/human_g1k_v37/human_g1k_v37_cas9.fasta'

In [4]:
genbankfile_url = 'https://raw.githubusercontent.com/biopython/biopython/master/Tests/GenBank/NC_005816.gb'
example_fastq_path = '/home/users/team_projects/PCNSL/WGS/22_new_fastq/LY02_S1L_R1.fastq.gz'
example_stockholm_url = 'http://pfam.xfam.org/family/PF05356/alignment/seed'

In [128]:
>>> from Bio.Seq import Seq
>>> from Bio.SeqRecord import SeqRecord
>>> record = SeqRecord(
...     Seq(
...         "MMYQQGCFAGGTVLRLAKDLAENNRGARVLVVCSEITAVTFRGPSETHLDSMVGQALFGD"
...         "GAGAVIVGSDPDLSVERPLYELVWTGATLLPDSEGAIDGHLREVGLTFHLLKDVPGLISK"
...         "NIEKSLKEAFTPLGISDWNSTFWIAHPGGPAILDQVEAKLGLKEEKMRATREVLSEYGNM"
...         "SSAC"
...     ),
...     id="gi|14150838|gb|AAK54648.1|AF376133_1",
...     description="chalcone synthase [Cucumis sativus]",
... )

In [132]:
print(record.format('fasta'))

>gi|14150838|gb|AAK54648.1|AF376133_1 chalcone synthase [Cucumis sativus]
MMYQQGCFAGGTVLRLAKDLAENNRGARVLVVCSEITAVTFRGPSETHLDSMVGQALFGD
GAGAVIVGSDPDLSVERPLYELVWTGATLLPDSEGAIDGHLREVGLTFHLLKDVPGLISK
NIEKSLKEAFTPLGISDWNSTFWIAHPGGPAILDQVEAKLGLKEEKMRATREVLSEYGNM
SSAC



In [180]:
with urllib.request.urlopen(genbankfile_url) as response:
    record = Bio.SeqIO.read(io.TextIOWrapper(response), 'genbank')

In [197]:
with urllib.request.urlopen(example_stockholm_url) as response:
#     print(gzip.open(response, mode='rt').read())
    alignment = Bio.AlignIO.read(io.TextIOWrapper(response), 'stockholm')

In [199]:
alignment[0]

SeqRecord(seq=Seq('LFMVTDAQAAALVAN------ITVTDIVDQLK--AGAPAIVSIATAAISLLAVI...GAL'), id='I3CK69_9GAMM/17-72', name='I3CK69_9GAMM', description='I3CK69_9GAMM/17-72', dbxrefs=[])

In [200]:
alignment

<<class 'Bio.Align.MultipleSeqAlignment'> instance (69 records of length 64) at 2ac4e8b53d00>

In [205]:
help(Bio.Align.Applications.MuscleCommandline)

Help on class MuscleCommandline in module Bio.Align.Applications._Muscle:

class MuscleCommandline(Bio.Application.AbstractCommandline)
 |  MuscleCommandline(cmd='muscle', **kwargs)
 |  
 |  Command line wrapper for the multiple alignment program MUSCLE.
 |  
 |  http://www.drive5.com/muscle/
 |  
 |  Notes
 |  -----
 |  Last checked against version: 3.7, briefly against 3.8
 |  
 |  References
 |  ----------
 |  Edgar, Robert C. (2004), MUSCLE: multiple sequence alignment with high
 |  accuracy and high throughput, Nucleic Acids Research 32(5), 1792-97.
 |  
 |  Edgar, R.C. (2004) MUSCLE: a multiple sequence alignment method with
 |  reduced time and space complexity. BMC Bioinformatics 5(1): 113.
 |  
 |  Examples
 |  --------
 |  >>> from Bio.Align.Applications import MuscleCommandline
 |  >>> muscle_exe = r"C:\Program Files\Alignments\muscle3.8.31_i86win32.exe"
 |  >>> in_file = r"C:\My Documents\unaligned.fasta"
 |  >>> out_file = r"C:\My Documents\aligned.fasta"
 |  >>> muscle_cl

# PairwiseAligner

In [9]:
import Bio.Align
from handygenome.align.realign import ALIGNER_BLASTN

ModuleNotFoundError: No module named 'pysam'

In [ ]:

print(Bio.Align.PairwiseAligner())

Pairwise sequence aligner with parameters
  wildcard: None
  match_score: 1.000000
  mismatch_score: 0.000000
  target_internal_open_gap_score: 0.000000
  target_internal_extend_gap_score: 0.000000
  target_left_open_gap_score: 0.000000
  target_left_extend_gap_score: 0.000000
  target_right_open_gap_score: 0.000000
  target_right_extend_gap_score: 0.000000
  query_internal_open_gap_score: 0.000000
  query_internal_extend_gap_score: 0.000000
  query_left_open_gap_score: 0.000000
  query_left_extend_gap_score: 0.000000
  query_right_open_gap_score: 0.000000
  query_right_extend_gap_score: 0.000000
  mode: global



In [9]:
def gapscore(start, length):
    if start == 0:
        return 2
    else:
        return 0

aligner = Bio.Align.PairwiseAligner(
    match_score=10,
    mismatch_score=-3,
    query_gap_score=gapscore
)

In [ ]:
aligner.align()

In [7]:
print(aligner)

Pairwise sequence aligner with parameters
  wildcard: None
  match_score: 1.000000
  mismatch_score: 0.000000
  target_internal_open_gap_score: 0.000000
  target_internal_extend_gap_score: 0.000000
  target_left_open_gap_score: 0.000000
  target_left_extend_gap_score: 0.000000
  target_right_open_gap_score: 0.000000
  target_right_extend_gap_score: 0.000000
  query_gap_function: <function gapscore at 0x2ae640564670>
  mode: global



In [30]:
target = 'CCCCCCCAAAAACCCCCCCAA'
query = 'CCCCCCC'

def gapscorer(start, length):
    return -7 + -2 * (length - 1)


def gapscorer_free_leftgap(start, length):
    if start == 0:
        return 0
    else:
        return -7 + -2 * (length - 1)


# aligner = Bio.Align.PairwiseAligner(
#     mode='global',
#     match_score=2,
#     mismatch_score=-3,
#     query_internal_open_gap_score=-7,
#     query_internal_extend_gap_score=-2,
#     target_internal_open_gap_score=-7,
#     target_internal_extend_gap_score=-2,

#     query_left_open_gap_score=0,
#     query_left_extend_gap_score=0,
#     query_right_open_gap_score=0,
#     query_right_extend_gap_score=0,
# )

aligner = Bio.Align.PairwiseAligner(
    mode='local',
    match_score=2,
    mismatch_score=-3, 
    target_internal_open_gap_score=-7,
    target_internal_extend_gap_score=-2,
    query_gap_score=gapscorer_free_leftgap,
    # query_gap_score=gapscorer,
)

alns = aligner.align(target, query)
print(*alns)

CCCCCCCAAAAACCCCCCCAA
|||||||
CCCCCCC
 CCCCCCCAAAAACCCCCCCAA
            |||||||
            CCCCCCC



In [31]:
alns[1].path

((12, 0), (19, 7))

In [32]:
type(alns[0])

Bio.Align.PairwiseAlignment

In [34]:
aln = alns[0]
new_path = [(len(aln.target) - x[0], len(aln.query) - x[1]) for x in aln.path[::-1]]
new_aln = Bio.Align.PairwiseAlignment(aln.target[::-1], aln.query[::-1], new_path, 0)

In [35]:
print(aln)
print(new_aln)

CCCCCCCAAAAACCCCCCCAA
|||||||
CCCCCCC

AACCCCCCCAAAAACCCCCCC
              |||||||
              CCCCCCC

